In [1]:
import os
from tkinter import Image
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
import torch
import PIL
from PIL import Image
from torchvision.transforms.functional import to_pil_image
import numpy as np
from torch.utils.data import DataLoader
from torchvision.io import read_image
from torchvision import transforms


In [2]:
read_image("C:\\Users\\user\\Desktop\\temp\\back\\0036_c2_f0062476.jpg").shape

torch.Size([3, 184, 73])

In [144]:

_target = np.array(['front', 'back', 'left', 'right'])

class TrainRawData(Dataset):
    def __init__(self, dir, img_size = (150, 80)):
        self.target = _target
        self.dir = dir
        self.img_size = img_size
        imgs = []
        labels = []
        for t in self.target:
            image_files = os.listdir(os.path.join(dir, t))
            for file in image_files:
                imgs += [os.path.join(dir, t, file)]
                labels += [t]

        self.images = imgs
        self.labels = labels
        self.resize = transforms.Resize(self.img_size)
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        img = read_image(self.images[index])        
        img = self.resize(img)
        label = self.labels[index]

        return img, label

dir = "C:\\Users\\user\\Desktop\\pose-angle"
dataset = TrainRawData(dir)
predictor = openpifpaf.Predictor(checkpoint='shufflenetv2k16')
skeletons = []
labels = []
dataloader = DataLoader(dataset, batch_size = 32, shuffle= False)

for batch, (X, y) in enumerate(dataloader):
    images =[]
    for img in X:
        images += [to_pil_image(img)]

    
    predicts = predictor.pil_images(images)
    

    for p, y in zip(predicts, y):
        if not p:
            continue
        if not p[0]:
            continue
        
        skel = p[0][0].data
        skel[:, 1:2] = skel[:, 1:2]/150
        skel[:, 0:1] = skel[:, 0:1]/50
        new_skel = (skel[5] + skel[6])/2
        skeleton = np.append(skel, new_skel[np.newaxis, :], axis = 0)
        skeletons += [skeleton]
        labels += [y]
        

KeyboardInterrupt: 

In [119]:
len(skeletons)

5

In [5]:
from PIL import Image
img = Image.open("C:\\Users\\user\\Desktop\\temp\\back\\0036_c2_f0062476.jpg")


In [6]:
img.size

(73, 184)

In [1]:
predictor = openpifpaf.Predictor(checkpoint='shufflenetv2k16')
skeletons = []
labels = []

target = np.array(['front', 'back', 'left', 'right'])


for k, (i, l) in enumerate(dataset):
    image = to_pil_image(i)
    predictions, _, _ = predictor.pil_image(image)
    label = target == l
    if not predictions:
        continue

    skt = np.append(predictions[0].data, (predictions[0].data[5] + predictions[0].data[6])/2).reshape(-1, 3)
    skt[:, 1:2] = skt[:, 1:2]/150
    skt[:, 0:1] = skt[:, 0:1]/80


    if len(skeletons) ==0:
        skeletons = skt[np.newaxis, :]
        labels = label[np.newaxis, :]
    else :
        skeletons = np.append(skeletons, skt[np.newaxis, :], axis = 0)
        labels = np.append(labels, label[np.newaxis, :], axis = 0)

# print('save file')
# np.save("./skeletons",skeletons)
# print('skeletons.npy done')
# np.save('./labels', labels)
# print('labels.npy done')

NameError: name 'openpifpaf' is not defined

In [140]:
skeletons

array([[[ 0.43082315,  0.12399478,  0.7198429 ],
        [ 0.46405753,  0.10532612,  0.74980545],
        [ 0.38267812,  0.10790325,  0.75062114],
        ...,
        [ 0.5836378 ,  0.8786189 ,  0.8731816 ],
        [ 0.38348863,  0.87021655,  0.8877806 ],
        [ 0.45100838,  0.21286431,  0.9859555 ]],

       [[-0.14      , -0.03333334,  0.        ],
        [-0.14      , -0.03333334,  0.        ],
        [-0.14      , -0.03333334,  0.        ],
        ...,
        [ 0.578619  ,  0.8178058 ,  0.8126712 ],
        [ 0.3860769 ,  0.83724546,  0.819913  ],
        [ 0.5202601 ,  0.16737348,  0.8793916 ]],

       [[ 0.43231776,  0.12139782,  0.9094761 ],
        [ 0.49593487,  0.10381035,  0.8891215 ],
        [ 0.39445677,  0.10155147,  0.8944227 ],
        ...,
        [ 0.5812443 ,  0.875796  ,  0.942882  ],
        [ 0.48856452,  0.8841891 ,  0.9106627 ],
        [ 0.46228835,  0.21491969,  0.96181136]],

       ...,

       [[ 0.3813506 ,  0.15464474,  0.7758648 ],
        [ 0

In [1]:
from datasets import AlphaposeSkeletonData

train_json_dir = "C:\\Users\\user\\Desktop\\alphapose_json"
train_image_dir = "C:\\Users\\user\\Desktop\\RE-ID\\datasets\\DukeMTMC-reID\\query"
valid_json_dir = "C:\\Users\\user\\Desktop\\alphapose_json_valid"
valid_image_idr =  "C:\\Users\\user\\Desktop\\RE-ID\\datasets\\DukeMTMC-reID\\query"

train_dataset = AlphaposeSkeletonData(train_json_dir, train_image_dir)
valid_dataset = AlphaposeSkeletonData(valid_json_dir, valid_image_idr)
train_dataset.__len__(), valid_dataset.__len__()

c:\Users\user\Desktop\GCN\datasets.py:194: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:210.)
  self.skeletons = torch.tensor(skeletons).float()


(1112, 128)

In [2]:
from train import train, _A
from model import GCN
import torch

model = GCN(3, 4, torch.tensor(_A).float())

train(model, train_dataset, valid_dataset, epochs= 100, batch_size=16)

learning_rate : 0.001
batch_size : 16
epochs : 100
Epoch 1
---------------------------------
loss: 4.671702 [    0/ 1112]
loss: 2.265353 [  100/ 1112]
loss: 1.183105 [  200/ 1112]
loss: 1.275441 [  300/ 1112]
loss: 1.473228 [  400/ 1112]
loss: 1.281156 [  500/ 1112]
loss: 1.342557 [  600/ 1112]
loss: 1.296551 [  700/ 1112]
loss: 1.111984 [  800/ 1112]
loss: 1.026335 [  900/ 1112]
loss: 1.270330 [ 1000/ 1112]
loss: 1.265838 [ 1100/ 1112]
val_loss: 19.374121
---------------------------------
Epoch 2
---------------------------------
loss: 1.295339 [    0/ 1112]
loss: 1.035140 [  100/ 1112]
loss: 1.285449 [  200/ 1112]
loss: 1.312152 [  300/ 1112]
loss: 1.255102 [  400/ 1112]
loss: 1.254688 [  500/ 1112]
loss: 1.169887 [  600/ 1112]
loss: 1.082679 [  700/ 1112]
loss: 1.235345 [  800/ 1112]
loss: 1.285422 [  900/ 1112]
loss: 1.220614 [ 1000/ 1112]
loss: 1.086609 [ 1100/ 1112]
val_loss: 20.949520
---------------------------------
Epoch 3
---------------------------------
loss: 1.086794 [   

In [17]:
import numpy as np
_target = np.array(['front', 'back', 'left', 'right'])

model.eval()
s = 0
with torch.no_grad():
    for X, y in valid_dataset:
        pred = model(X.unsqueeze(0))
        s += int(_target[pred.argmax()]== _target[y.argmax()])

print("accruacy : ", s/valid_dataset.__len__())


accruacy :  0.8359375


In [ ]:
with torch.no_grad():
    pred = model(X)
    print(_target[pred.argmax()], _target[y.argmax()])
